# A real world workchain example: electronic band structure


*Import statements - make sure to execute the cell below this one (it may be hidden)*

In [ ]:
%matplotlib inline
%aiida

from datetime import datetime, timedelta

from aiida.tools.dbimporters.plugins.cod import CodDbImporter
from aiida.engine import launch

PwBandStructureWorkChain = WorkflowFactory('quantumespresso.pw.band_structure')

### Calculating the electronic band structure with an AiiDA workchain
This tutorial will show how useful a workchain can be in performing a well defined task, such as computing and visualizing the electronic band structure for a simple crystal structure. The goal of this tutorial is not to show you the intricacies of the actual workchain itself, but rather to serve as an example that workchains can simplify standard workflows in computational materials science. The workchain that we will use here will employ Quantum Espresso's pw.x code to calculate the charge densities for several crystal structures and compute a band structure from those. Many choices that normally face the researcher before being able to perform this calculation, such as the selection of suitable pseudo potentials, energy cutoff values, k-point grids and k-point paths along high symmetry points, are now performed automatically by the workchain. All that remains for the user to do is to simply define a structure, pass it to the workchain and sit back!

Below, we import the crystal structure of Al as an example, and run the PwBandStructureWorkChain for that structure. The estimated run time is noted in a comment in the calculation cell.

In [ ]:
# Loading the COD importer so we can directly import structure from COD id's
importer = CodDbImporter()
# Make sure here to define the correct codename that corresponds to the pw.x code installed on your machine of choice
codename = 'qe-6.3-pw@localhost'
code = Code.get_from_string(codename)

#### Importing example crystal structures from COD to AiiDA structure objects

In [ ]:
# Al COD ID='9008460'
structure_Al = importer.query(id='9008460')[0].get_aiida_structure()

structure_Al.get_formula()

# The following structure can be used instead of Al, but will take much longer on the AWS machine.
# CaF2 COD ID='1000043' -- approximately 1/2 hour to run
# h-BN COD ID='9008997' -- approximately 45 mins to run
# GaAs COD ID='9008845' -- approximately 2 hours to run 

#### Now we run the bandstructure workchain for the selected structures

The bandstructure workchain follows the following protocol:
* Determine the primitive cell of the input structure
* Run a vc-relax to relax the structure
* Refine the symmetry of the relaxed structure to ensure the primitive cell is used and run a self-consistent field calculation on it
* Run a non self-consistent field band structure calculation along a path of high symmetry k-points determined by [seekpath](http://materialscloud.org/tools/seekpath)

Numerical parameters for the default 'theos-ht-1.0' protocol are determined as follows:
* Suitable pseudopotentials and energy cutoffs are automatically searched from the [SSSP library], I(http://materialscloud.org/sssp) installed on your machine  (it uses the efficiency version 1.1).
* K-point mesh is selected to have a minimum k-point density of 0.2 Å<sup>-1</sup>
* A Marzari-Vanderbilt smearing of 0.02 Ry is used for the electronic occupations

If the pseudopotentials are not installed, they can be downloaded in a terminal as:

    wget https://archive.materialscloud.org/file/2018.0001/v1/SSSP_efficiency_pseudos.tar.gz
    tar -zxvf SSSP_efficiency_pseudos.tar.gz
    
The protocol looks for a UpF file with a specific hash code, that is unique for each different file. 
You can check that you have the right
one by performing a search in the database:

    qb=QueryBuilder()
    qb.append(UpfData, filters={'attributes.md5':{'==':'cfc449ca30b5f3223ec38ddd88ac046d'}})
    len(qb.all())

'md5' is a searchable attribute of the pseudopotential data object.

In [ ]:
# This will take approximately 6 minutes on the tutorial AWS (for Al)
results = launch.run(
    PwBandStructureWorkChain,
    code=code,
    structure=structure_Al
)

In [ ]:
fermi_energy = results['scf_parameters'].dict.fermi_energy
results['band_structure'].show_mpl(y_origin=fermi_energy, plot_zero_axis=True)

print("""Final crystal symmetry: {spacegroup_international} (number {spacegroup_number})
Extended Bravais lattice symbol: {bravais_lattice_extended}
The system has inversion symmetry: {has_inversion_symmetry}""".format(
    **results['seekpath_parameters'].get_dict()))

If the pseudopotential family that you downloaded is the same efficiency pseudopotential family, but an older version  (the https://archive.materialscloud.org/file/2018.0001/v1/SSSP_efficiency_pseudos.tar.gz -- note the v1 in the link -- you can use the PwBandStructureWorkChain can be used, by providing a modification of the existing protocol

In [ ]:
# This will take approximately 6 minutes on the tutorial AWS (for Al)
results = launch.run(
    PwBandStructureWorkChain,
    code=code,
    structure=structure_Al,
    protocol=Dict(dict={
        'name':'theos-ht-1.0',
        'pseudo':'SSSP-efficiency-1.0'
    })
)